In [1]:
# How to get the kernel working...
"""
mamba create -n torchtune python=3.12
mamba activate torchtune
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/rocm6.2
pip install torchao
pip install torchtune
pip install ipykernel nbformat
python -m ipykernel install --user --name=torchtune
"""
# Select in the top right

'\nmamba create -n torchtune python=3.12\nmamba activate torchtune\npip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/rocm6.2\npip install ipykernel nbformat\npython -m ipykernel install --user --name=torchtune\n'

In [8]:
# https://github.com/pytorch/torchtune?tab=readme-ov-file#get-started
!tune download meta-llama/Meta-Llama-3.1-8B-Instruct --ignore-patterns none --output-dir /tune/Meta-Llama-3.1-8B-Instruct

Ignoring files matching the following patterns: none
Fetching 17 files: 100%|█████████████████████| 17/17 [00:00<00:00, 23045.63it/s]
Successfully downloaded model repo and wrote to the following locations:
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/hf_model_0001_0.pt
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/model-00004-of-00004.safetensors
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/model-00001-of-00004.safetensors
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/model-00002-of-00004.safetensors
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/special_tokens_map.json
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/original
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/tokenizer.json
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/config.json
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/model.safetensors.index.json
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/model-00003-of-00004.safetensors
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/generation_config.json
/mnt/nvme2n1p1/Meta-Llama-3.1-8B-Instruct/USE_POLICY.md
/mnt/nvme2n1

In [13]:
!tune ls

RECIPE                                   CONFIG                                  
full_finetune_single_device              llama2/7B_full_low_memory               
                                         code_llama2/7B_full_low_memory          
                                         llama3/8B_full_single_device            
                                         llama3_1/8B_full_single_device          
                                         llama3_2/1B_full_single_device          
                                         llama3_2/3B_full_single_device          
                                         mistral/7B_full_low_memory              
                                         phi3/mini_full_low_memory               
                                         qwen2/7B_full_single_device             
                                         qwen2/0.5B_full_single_device           
                                         qwen2/1.5B_full_single_device           
                

In [ ]:
!time tune run full_finetune_single_device --config tt-llama3_1/8B_full_single_device.yaml

INFO:torchtune.utils._logging:Running FullFinetuneRecipeSingleDevice with resolved config:

batch_size: 2
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tune/Meta-Llama-3.1-8B-Instruct/
  checkpoint_files:
  - model-00001-of-00004.safetensors
  - model-00002-of-00004.safetensors
  - model-00003-of-00004.safetensors
  - model-00004-of-00004.safetensors
  model_type: LLAMA3
  output_dir: /tune/Meta-Llama-3.1-8B-Instruct/
  recipe_checkpoint: null
compile: false
dataset:
  _component_: torchtune.datasets.alpaca_dataset
device: cuda
dtype: bf16
enable_activation_checkpointing: true
epochs: 3
gradient_accumulation_steps: 1
log_every_n_steps: 1
log_peak_memory_stats: false
loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss
max_steps_per_epoch: null
metric_logger:
  _component_: torchtune.training.metric_logging.DiskLogger
  log_dir: /tune/full-llama3.1-finetune
model:
  _component_: torchtune.models.llama3_1.llama3_1_8b
optimizer:


In [19]:
!time tune run --nnodes 1 --nproc_per_node 8 full_finetune_distributed --config tt-llama3_1/8B_full.yaml

Running with torchrun...
W1023 06:01:55.491000 858110 /mnt/nvme1n1p1/miniforge3/envs/torchtune/lib/python3.12/site-packages/torch/distributed/run.py:793] 
W1023 06:01:55.491000 858110 /mnt/nvme1n1p1/miniforge3/envs/torchtune/lib/python3.12/site-packages/torch/distributed/run.py:793] *****************************************
W1023 06:01:55.491000 858110 /mnt/nvme1n1p1/miniforge3/envs/torchtune/lib/python3.12/site-packages/torch/distributed/run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1023 06:01:55.491000 858110 /mnt/nvme1n1p1/miniforge3/envs/torchtune/lib/python3.12/site-packages/torch/distributed/run.py:793] *****************************************
INFO:torchtune.utils._logging:Running FullFinetuneRecipeDistributed with resolved config:

batch_size: 2
checkpointer:
  _component_: torchtune.training.FullModelHF

In [21]:
!time tune run --nnodes 1 --nproc_per_node 8 full_finetune_distributed --config tt-llama3_1/8B_full.yaml

Running with torchrun...
W1023 07:21:31.612000 955787 /mnt/nvme1n1p1/miniforge3/envs/torchtune/lib/python3.12/site-packages/torch/distributed/run.py:793] 
W1023 07:21:31.612000 955787 /mnt/nvme1n1p1/miniforge3/envs/torchtune/lib/python3.12/site-packages/torch/distributed/run.py:793] *****************************************
W1023 07:21:31.612000 955787 /mnt/nvme1n1p1/miniforge3/envs/torchtune/lib/python3.12/site-packages/torch/distributed/run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1023 07:21:31.612000 955787 /mnt/nvme1n1p1/miniforge3/envs/torchtune/lib/python3.12/site-packages/torch/distributed/run.py:793] *****************************************
INFO:torchtune.utils._logging:Running FullFinetuneRecipeDistributed with resolved config:

batch_size: 16
checkpointer:
  _component_: torchtune.training.FullModelH